In [62]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd

In [63]:
def scrapeItems(urlList, itemName):
    itemDf = pd.DataFrame(columns=['Brand', 'Price', 'Image', 'Length', 'Width', 'Height', 'Seller', 'ProductLink'])

#     urlList = ["https://www.ikea.com/us/en/p/friheten-sleeper-sectional-3-seat-w-storage-skiftebo-blue-s69297560/", "https://www.ikea.com/us/en/p/karlstad-sofa-knisa-light-gray-s39132702/", "https://www.ikea.com/us/en/p/kivik-sofa-hillared-with-chaise-hillared-anthracite-s69193666/", "https://www.ikea.com/us/en/p/hemnes-daybed-with-3-drawers-2-mattresses-white-meistervik-firm-s59186119/", "https://www.ikea.com/us/en/p/knislinge-sofa-idhult-black-80278923/", "https://www.ikea.com/us/en/p/faerloev-sleeper-sofa-flodafors-white-s09252753/", "https://www.ikea.com/us/en/p/sandbacken-sleeper-sectional-3-seat-lofallet-beige-s89217906/"]
    for url in urlList:
        #url = 'https://www.ikea.com/us/en/p/landskrona-loveseat-gunnared-dark-gray-metal-s89270277/'
        print(url)
        response = requests.get(url)
#         response
        soup = BeautifulSoup(response.text, "html.parser")
        brand = soup.findAll("span", {"class":"product-pip__name"})[0].contents[0]
        price = soup.findAll("span", {"class":"product-pip__price__value"})[0].contents[0]
        #Contains all dimension-names from product description
        dimensions_desc = soup.findAll("dt", {"class":"product-pip__definition-list-item"})

        imgLink = soup.findAll("img", {"class":""})[0]['src']
        #Contains all dimension-values from product description
        dimensions = soup.findAll("dd", {"class":"product-pip__definition-list-item"})
        print(dimensions_desc)
        print(dimensions)
        lStr="length"
        wStr = "width"
        hStr = "height"
        #L,W,D
        if ("length" in dimensions_desc or "Length" in dimensions_desc) and ("width" in dimensions_desc or "Width" in dimensions_desc) and ("depth" in dimensions_desc or "Depth" in dimensions_desc):
            hStr="depth"
        #H,W,D
        elif ("height" in dimensions_desc or "Height" in dimensions_desc) and ("width" in dimensions_desc or "Width" in dimensions_desc) and ("depth" in dimensions_desc or "Depth" in dimensions_desc):
            lStr="depth"
        #L,D,H
        elif ("length" in dimensions_desc or "Length" in dimensions_desc) and ("depth" in dimensions_desc or "Depth" in dimensions_desc) and ("height" in dimensions_desc or "Height" in dimensions_desc):
            wStr="depth"
        print(lStr,wStr,hStr)
        #Finding All height-related dimensions
        length_index = []
        for dim in range(0, len(dimensions_desc)):
            if(lStr in dimensions_desc[dim].contents[0].lower()): #height
                length_index.append(dim)

        #Finding max of given heights. Seemed in sync with most of the images 
        #with minute variations in the exceptions.
        lengths = []
        for index in length_index:
            lengths.append(dimensions[index].contents[0])

        length = max(lengths)
        print(length)

        width_index = []
        for dim in range(0, len(dimensions_desc)):
            if(wStr in dimensions_desc[dim].contents[0].lower()):
                width_index.append(dim)
        widths = []
        for index in width_index:
            widths.append(dimensions[index].contents[0])

        width = max(widths)
        print(width)

        height_index = []
        for dim in range(0, len(dimensions_desc)):
            if(hStr in dimensions_desc[dim].contents[0].lower()):
                height_index.append(dim)
        heights = []
        for index in height_index:
            heights.append(dimensions[index].contents[0])

        height = max(heights)
        print(height)

        productLink=url
        print(brand, price, imgLink, length, width, height, productLink)
        itemDf = itemDf.append({"Brand":brand, "Price":price, "Image":imgLink, "Length":length, "Width":width, "Height":height, "Seller":"IKEA", "ProductLink":productLink}, ignore_index=True)

#     itemDf
    
    filename = itemName+".csv"
    itemDf.to_csv(filename)

In [64]:
fourProducts = pd.read_csv("FourProductsURL.csv")
noOfItems = len(fourProducts)
items = list(fourProducts)
print(items)
for itemNumber in range(len(items)):
#     print(items[itemNumber])
    itemDf=fourProducts[items[itemNumber]].dropna()
    print(itemDf)
    scrapeItems(itemDf,items[itemNumber])

['desk', 'table', 'mattress', 'sofa']
0     https://www.ikea.com/us/en/p/idasen-desk-sit-s...
1     https://www.ikea.com/us/en/p/alex-desk-beige-3...
2     https://www.ikea.com/us/en/p/bekant-desk-white...
3     https://www.ikea.com/us/en/p/hemnes-desk-black...
4     https://www.ikea.com/us/en/p/knotten-standing-...
5     https://www.ikea.com/us/en/p/pahl-desk-white-s...
6     https://www.ikea.com/us/en/p/pahl-desk-white-s...
7     https://www.ikea.com/us/en/p/ingatorp-desk-whi...
8     https://www.ikea.com/us/en/p/arkelstorp-desk-b...
9     https://www.ikea.com/us/en/p/micke-desk-black-...
10    https://www.ikea.com/us/en/p/idasen-desk-brown...
11    https://www.ikea.com/us/en/p/bekant-screen-for...
12    https://www.ikea.com/us/en/p/micke-desk-black-...
13    https://www.ikea.com/us/en/p/bekant-desk-sit-s...
14    https://www.ikea.com/us/en/p/lisabo-desk-ash-v...
15    https://www.ikea.com/us/en/p/bekant-desk-black...
16    https://www.ikea.com/us/en/p/hemnes-desk-with-...
17    http

ValueError: max() arg is an empty sequence